# Sacred Experiment Comparison

The goal of this notebook is to provide a mechanism to compare sacred experiments: the cfg, the metrics, and the outcome.

In [77]:
# project imports
import env
from utils import read_json_data, flatten_dict
from models import MODEL_DIR


# python & package imports
from collections import defaultdict
import pandas as pd
import json
import os


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2


model_dir = os.path.join(MODEL_DIR, 'voxel_vaegan1', 'modelnet10')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Model Leaderboard

In [69]:
scores = dict()
for m in os.listdir(model_dir):
    name = m
    metrics_json = os.path.join(model_dir, m, 'metrics.json')
    #print(metrics_json)
    #if os.path.exists(metrics_json):
    #    df = pd.read_json(metrics_json, orient='values')
    #    break
    metrics_dict = read_json_data(metrics_json)
    if metrics_dict:
        # assuming only keys are epoch\d+
        keys = [int(key.replace('epoch', '')) for key in metrics_dict.keys()]
        last_epoch = sorted(keys)[-1]
        last_key = 'epoch{}'.format(last_epoch)
        scores[name] = metrics_dict[last_key]
        scores[name]['epoch'] = last_epoch

df = pd.DataFrame.from_dict(scores, orient='index')

In [70]:
df.sort_values('enc_loss')

,dis_loss,reconstruction_loss,kl_divergence,epoch,dec_loss,enc_loss,elapsed_time
best,-999.0,808.499634,488.078369,9,-999.0,261.985138,43.024081
2019-03-14_16-33-24,-999.0,3353.308350,593.102234,9,-999.0,302.278900,36.155712
2019-03-14_15-51-00,-999.0,3414.346680,580.177063,9,-999.0,305.875122,35.956132


## Config Comparison

In [85]:
# provide the full path to root dir of the experiment output (the parent dir of the ckpts)
exp1 = os.path.join(model_dir, 'best')
exp2 = os.path.join(model_dir, '2019-03-14_15-51-00')

In [90]:
cfg1 = read_json_data(os.path.join(exp1, 'cfg.json'), verbose=True)
cfg2 = read_json_data(os.path.join(exp2, 'cfg.json'), verbose=True)
assert isinstance(cfg1, dict)
assert isinstance(cfg2, dict)

# flatten dict structure then bring into pandas for a pretty view
cfg1 = flatten_dict(cfg1)
cfg2 = flatten_dict(cfg2)
df1 = pd.DataFrame.from_dict(cfg1, orient='index', columns=['cfg1'])
df2 = pd.DataFrame.from_dict(cfg2, orient='index', columns=['cfg2'])
dff = pd.concat([df1, df2], axis=1, sort=False)
dff['diff'] = dff.cfg1 == dff.cfg2
dff

,cfg1,cfg2,diff
model/ckpt_dir,voxel_vaegan1/2019-03-12_16-36-25,/home/jcworkma/jack/3d-form/src/../models/voxe...,False
model/dis_lr,0.0001,NaN,False
model/tb_dir,tb,tb,True
model/no_gan,True,True,True
model/debug,False,False,True
generator/verbose,True,True,True
model/recon_loss_weight,10000,10000,True
model/save_step,10,5,False
model/dev_step,10,2,False
model/voxel_prob_threshold,0.065,0.065,True


## Just in case you need to retrieve a ckpt_dir from an old sacred config.json save

In [10]:
root = '/home/jcworkma/jack/3d-form/notebooks/experiments_vaegan'
for exp_dir in os.listdir(root):
    if exp_dir == '_sources':
        continue
    #print('-------')
    #print(exp_dir)
    cfg_path = os.path.join(root, exp_dir + '/config.json')
    with open(cfg_path, 'r') as json_file:
        cfg = json.load(json_file)
    ckpt_dir = cfg.get('cfg', 'banana').get('model', 'apple').get('ckpt_dir', 'orange')
    #print(ckpt_dir)
    if '2019-03-12_16-36-25' in ckpt_dir:
        print('found it!')
        print(cfg_path)
        print(ckpt_dir)
        break

found it!
/home/jcworkma/jack/3d-form/notebooks/experiments_vaegan/99/config.json
voxel_vaegan1/2019-03-12_16-36-25
